In [1]:
from __future__ import print_function, division

# Import GDAL, NumPy, and matplotlib
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
%matplotlib inline

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()


in_directory = r"R:/Work/medieval 2017/FDSI Dev/allofthem"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:463]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [2]:
in_directory = r"R:/Work/medieval 2017/FDSI Dev/allofthem_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:463]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [3]:
X=np.vstack(images)

In [4]:
Y=np.vstack(masks)

In [5]:
# Find how many non-zero entries we have -- i.e. how many training data samples?
n_samples = (Y>=0).sum()
print('We have {n} samples'.format(n=n_samples))

# What are our classification labels?
labels = np.unique(roi)
print('The training data include {n} classes: {classes}'.format(n=labels.size, 
                                                                classes=labels))
# We will need a "X" matrix containing our features, and a "y" array containing our labels
#     These will have n_samples rows
#     In other languages we would need to allocate these and them loop to fill them, but NumPy can be faster

s = X[Y >= 0] 
l = Y[Y >= 0]

print('Our X matrix is sized: {sz}'.format(sz=s.shape))
print('Our y array is sized: {sz}'.format(sz=l.shape))


print('After masking, our X matrix is sized: {sz}'.format(sz=s.shape))
print('After masking, our y array is sized: {sz}'.format(sz=l.shape))

We have 47308800 samples
The training data include 2 classes: [0 1]
Our X matrix is sized: (47308800, 4)
Our y array is sized: (47308800,)
After masking, our X matrix is sized: (47308800, 4)
After masking, our y array is sized: (47308800,)


In [6]:
from sklearn.svm import LinearSVC
SVM = LinearSVC(dual=False, random_state=0, max_iter=1000)
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=5,max_depth=2,max_features=None, random_state=0)
from sklearn.naive_bayes import GaussianNB
MLC = GaussianNB()
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('SVM', SVM), ('RF', RF), ('MLC', MLC)], voting='hard')

In [7]:
ECLF = eclf.fit(s,l)

In [8]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_01_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:100]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [9]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [10]:
test_result = ECLF.predict(test_pixels)

In [11]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_01_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:100]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [12]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(T, test_result)

0.68582440776209674

In [19]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_02_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:50]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [20]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [21]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_02_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:50]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [22]:
test_result = ECLF.predict(test_pixels)

In [23]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [24]:
accuracy_score(T, test_result)

0.59622985839843745

In [25]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_03_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:70]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [26]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [27]:
test_result = ECLF.predict(test_pixels)

In [30]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_03_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:70]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [31]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [32]:
accuracy_score(T, test_result)

0.6217258673199153

In [33]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_04_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:70]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [34]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [35]:
test_result = ECLF.predict(test_pixels)

In [36]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_04_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:70]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [37]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [38]:
accuracy_score(T, test_result)

0.87694213867187498

In [39]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_05_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:70]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [40]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [41]:
test_result = ECLF.predict(test_pixels)

In [42]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_05_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:70]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [43]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [44]:
accuracy_score(T, test_result)

0.68902762276785712

In [45]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_06_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:70]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [46]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [47]:
test_result = ECLF.predict(test_pixels)

In [48]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_06_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:70]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [49]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [50]:
accuracy_score(T, test_result)

0.85194069602272726

In [51]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_07_satellite_images"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:70]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [52]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [53]:
test_result = ECLF.predict(test_pixels)

In [54]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/testset_07_segmentation_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:70]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [55]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [56]:
accuracy_score(T, test_result)

0.66907075026939655

In [57]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/1to6"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))
files = files_to_process[0:205]
images = []
for f in files:
    raster_dataset = gdal.Open(f, gdal.GA_ReadOnly)
    img = np.zeros((raster_dataset.RasterYSize, raster_dataset.RasterXSize, raster_dataset.RasterCount),dtype=gdal_array.GDALTypeCodeToNumericTypeCode(raster_dataset.GetRasterBand(1).DataType))
    for b in range(raster_dataset.RasterCount):
        band = raster_dataset.GetRasterBand(b + 1)
        img[:, :, b] = band.ReadAsArray()
    if files_to_process is not None:
        images.append(img)

In [58]:
TX =np.vstack(images)
test_pixels = np.vstack(TX)

In [59]:
test_result = ECLF.predict(test_pixels)

In [60]:
in_directory = r"R:/Work/medieval 2017/FDSI Test/test_set/1to6_masks"
files_to_process_mask = glob.glob(os.path.join(in_directory, '*.png'))
files_mask = files_to_process_mask[0:205]
masks = []
for m in files_mask:
    roi_ds = gdal.Open(m, gdal.GA_ReadOnly)
    roi = roi_ds.GetRasterBand(1).ReadAsArray()
   
    if files_to_process_mask is not None:
        masks.append(roi)

In [61]:
TY=np.vstack(masks)
T=np.concatenate(TY, axis=0)

In [62]:
accuracy_score(T, test_result)

0.6626517539449257